In [ ]:
# Collection of demonstrations of PixelArray(PowerFlow) functionality

In [ ]:
push!(LOAD_PATH, "../jlfiles/")

using Plots, PixelArray, PixelArrayPowerFlow

pyplot()

In [ ]:
# Plot elliptic curve
# This doesn't rely on any multiplication or PAPF tests
genericport = Port(-2,2,100)
packXY = Pack(Dict("x"=>genericport, "y"=>genericport), ["x","y"])
demopwa = PackWithArray(packXY, pazeroset(packXY, (x,y) -> x^2 + y^3 - 0.5, 0.1))
hmp = plot(demopwa, color=cgrad([:white; :black]))

In [ ]:
# Plot half a line in an unnecessarily-complicated way to test out multiplication
# No PAPF required
genericport = Port(-2, 2, 100)
tol = 0.08
packXY = Pack(Dict("x"=>genericport, "y"=>genericport), ["x", "y"])
packYZ = Pack(Dict("y"=>genericport, "z"=>genericport), ["y", "z"])
pwaXY = PackWithArray(packXY, pazeroset(packXY, (x, y) -> x - y^2, tol))
pwaYZ = PackWithArray(packYZ, pazeroset(packYZ, (y, z) -> z - 2 * y^2, tol))
pwaXZ = mergemult([pwaXY, pwaYZ], ["y"])
hmp = plot(pwaXZ, color=cgrad([:white; :black]))

In [ ]:
# Plot a parabola with a single fixed parameter to test pagraph with fixed variables
genericport = Port(-2, 2, 100)
aval = -1.0
packXY = Pack(Dict("x"=>genericport, "y"=>genericport), ["x", "y"])
pwaXY = PackWithArray(packXY, pagraph(packXY, (x, a) -> x^2 + a, "y", Dict(3=>aval)))
hmp = plot(pwaXY, color=cgrad([:white; :black]))

In [ ]:
# Simple 2-bus problem
# Only exposed vars are P1, Q1

minθ = -π/6
maxθ = π/6

n = importnetwork("../testnetworks/2busfree", 1., minθ, maxθ)

allres = 100
tol = 0.05

psp = PackSetupParams(fill(allres,4)...)
@time pwas = getpfpwas(n, psp, tol)

# Intermediate: hide P2 and Q2
hidden = hidePQ(n, pwas, [2])
    
# Perform final array multiplication, leaving V2 and θ2 unexposed
finalpwa = clustertree(n, hidden, 1)

plot(finalpwa, color=cgrad([:white; :black]))

In [ ]:
# Experiment plotting run-time versus resolution for 2-bus system

# Setup
n = importnetwork("../testnetworks/2busfree", 1.)
tol = 0.1
ntrials = 30
resolutions = 1:ntrials
times = zeros(ntrials)

# Experiment, stage 1 (should scale with r^4)
for i = 1:ntrials
    psp = PackSetupParams(fill(resolutions[i],4)...)
    pwas = getpfpwas(n, psp, tol)
    
    times[i] = @elapsed begin
        # Intermediate: hide P2 and Q2
        hidden = hidePQ(n, pwas, [2])
        
        # Perform final array multiplication, leaving V2 and θ2 unexposed
        finalpwa = clustertree(n, hidden, 1)
    end
end

# More setup for stage 2
times2 = zeros(ntrials)

# Experiment, stage 2 (should scale with r^5)
for i = 1:ntrials
    psp2 = PackSetupParams(fill(resolutions[i],4)...)
    pwas2 = getpfpwas(n, psp2, tol)
    
    times2[i] = @elapsed begin
        # Hide P2 but not Q2
        hiddenP2 = mergemult([pwas2[3]], ["P2"])
        
        # Multiply all 4, leaving V2 and θ2 unexposed
        bigpwa = clustertree(n, [pwas2[1], pwas2[2], hiddenP2, pwas2[4]], 1)
        
        # Hide P2, Q2
        finalpwa2 = mergemult([bigpwa],["Q2"])
    end
end

In [ ]:
# Plot results
plot2 = plot(resolutions, times, seriescolor=:blue, label="Efficient Clustering")
plot!(plot2, resolutions, times2, seriescolor=:red, label="Inefficient Clustering")
plot!(plot2, xlabel="Resolution of each dimension", ylabel="Time (s)", legend=:topleft)
plot!(plot2, title="Multiplication Time vs. Resolution of 2-Bus Pixel Array")
#savefig(plot2, "complexity.pdf")